In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt
import nlopt
from IPython import display

from phodex.io import filter_stdout
from phodex.layout.meep import MultiportDevice2D, Port
from phodex.optim.callbacks import combine, logging_callback, plotting_callback
from phodex.optim.nlopt import get_epigraph_formulation
from phodex.topopt.parametrizations import sigmoid_parametrization

First we will define some basic variables such as resolution, materials, and design region size.

In [ ]:
wvg_width = 0.5
n_core = 3.4865
n_clad = 1.4440
wvg_height = 0.22
resolution = 20
wavelengths = np.linspace(1.5, 1.6, 5)
design_region = [3, 1.5]

Now we set up our device which, in the case of a waveguide, simply consists of an input an an output port.
Because we are specifying a waveguide height, `MultiportDevice2D` will assume that we want to run effective index simulations and will do the effective index calculation automatically.

In [ ]:
ports = [Port(wvg_width, "-x", source=True), Port(wvg_width, "+x")]

p = MultiportDevice2D(
    ports=ports,
    n_core=n_core,
    n_clad=n_clad,
    wvg_height=wvg_height,
    wavelengths=wavelengths,
    resolution=resolution,
    design_region_extent=design_region,
    monitor_size_fac=6,
    damping=1e-2,
)

Let's make sure that we got the geometry right!

In [ ]:
p.simulation.plot2D()

Looks good.

To define our objective functions, we will first need to get the input normalization, which we get by querying our `MultiportDevice2D`. The first time we ask for the normalization, this will trigger a normalization run with a straight waveguide (and no design region).

In [ ]:
input_flux_far = p.normalizations[0]["flux_far"]
input_flux_near = p.normalizations[0]["flux_near"]

Now, we can proceed to define our objective function. It takes the S-parameters of each port (propagating _towards_ the port) as arguments.
In this case, we will want to maximize the transmission in the output port, and we can do this by minimizing the total loss, i.e. reflection and whatever is not coupled into the fundamental mode of the output port.
Of course we can also simply maximize the `s12` parameter, which would probably work fine by itself. But since we have the reflection (`s11`) available, we might as well make use of it!

In [ ]:
def loss(s11, s12):
    return 1 - np.abs(s12) ** 2 / input_flux_far

def refl(s11, s12):
    refl = np.abs(s11) ** 2 / input_flux_near
    return refl

Now, let's assemble the Meep optimization problem...

In [ ]:
obj_funs = [loss, refl]
mpa_opt = p.get_optimization_problem(obj_funs)

...and check that all monitors are placed correctly.

In [ ]:
mpa_opt.plot2D(True)

The last ingredient we need is the parametrization, i.e. the mapping of our design variables (the degrees of freedom in the optimization) to the actual device layout.

Here, we will use a simple "sigmoid" parametrization that is often encountered in literature, where the material is first low-pass filtered to restrict the minimum feature size and then soft-thresholded using a (relatively steep) logistic function.

Note that we pick a simple set of fixed parameters here and will not dwell on feature size constraints and binarization. We will tackle these topics another time.

In [ ]:
parametrization = sigmoid_parametrization((p.nx, p.ny), sigma=4, alpha=20)

Before we proceed to the actual optimization, we will define a few callback functions that will let us track the optimization progress.

In [ ]:
state_dict = {"obj_hist": [], "epivar_hist": [], "cur_iter": 0}
log_cb = logging_callback(state_dict, logscale=True)

fig = plt.figure(figsize=(9, 6), tight_layout=True)
plot_cb = plotting_callback(mpa_opt, p, state_dict, figure=fig)


def notebook_updater(*_):
    display.display(fig)
    display.clear_output(wait=True)

Now, we can start to set up the actual optimization. Because we are doing an epigraph optimization, we will need a "dummy" variable (representing the linear objective function) in addition to the actual design variables.

We will also need to initialize the dummy variable, which we do by running a single forward pass of our optimization problem and setting the epigraph variable to a value slightly above the largest value across all objective functions.

In [ ]:
n = p.nx * p.ny + 1
x0 = np.full(n, 0.5)
lb = np.zeros_like(x0)
ub = np.ones_like(x0)

t0, _ = mpa_opt([parametrization(x0[1:])], need_gradient=False)
x0[0] = 1.05 * np.max(t0)
lb[0] = -np.inf
ub[0] = np.inf

We will also need to get the actual epigraph constraints for our problem, which `phodex` conveniently provides.

In [ ]:
nlopt_obj, epi_cst = get_epigraph_formulation(
    mpa_opt, parametrization, combine(log_cb, plot_cb, notebook_updater)
)
epi_tol = np.full(len(obj_funs) * p.nfreq, 1e-4)

And now we are finally ready to initialize the optimization! We will use the method of moving asymptotes (MMA) and limit the maximum number of iterations to 100.

In [ ]:
opt = nlopt.opt(nlopt.LD_MMA, n)
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)
opt.set_min_objective(nlopt_obj)
opt.add_inequality_mconstraint(epi_cst, epi_tol)
opt.set_param("dual_ftol_rel", 1e-7)
opt.set_maxeval(100)

And we're off!

In [ ]:
with filter_stdout("phodex"):
    x0[:] = opt.optimize(x0)